In [1]:
import os
import shutil

import pandas as pd
from salespy import SalesforceClass, EpicorClass

# login to Salesforce & get credentials
sf = SalesforceClass(
    username='bko@fs-elliott.com.test', sandbox='test',
    org_id='00D7A0000009Kmw'
)
creds = sf.login()
print(f'--- Logged Into Salesforce ---')

# epicor file directory
directory = (
    '/Users/beekayokay/OneDrive/Projects/'
    'Salesforce Epicor Integration/E10 Reports To Upload'
)

# loop through epicor files and merge
count = 0
epicor = EpicorClass()
bookings_df = pd.DataFrame()
for each in os.listdir(directory):
    temp_df = pd.DataFrame()
    if each == '.DS_Store':
        continue
    try:
        temp_df = epicor.bookings_to_df(os.path.join(directory, each))
        bookings_df = bookings_df.append(temp_df, ignore_index=True)
    except IsADirectoryError:
        continue
bookings_df.sort_values(by=['Name'], ascending=True, inplace=True)
bookings_df.drop_duplicates(inplace=True, keep='first')

--- Logged Into Salesforce ---


In [5]:
bookings_df.shape

(4631, 44)

In [6]:
# get most recent Tran Num for filtering Epicor report
last_tran_num = sf.get_last_tran(creds=creds)
print(f'--- Got Last Tran Num: {last_tran_num} ---')

# remove existing SFDC Tran Nums
old_trans = bookings_df['Name'] <= last_tran_num
bookings_df = bookings_df[~old_trans]
print(f'--- Removed Existing Epicor Data: {bookings_df.size} ---')

--- Got Last Tran Num: 0 ---
--- Removed Existing Epicor Data: 203764 ---


In [7]:
bookings_df.shape

(4631, 44)

In [8]:
# get tuples of Order Num Ln
order_num_ln = tuple(bookings_df['Order_Num_Ln__c'])
print(f'--- Compiled Order Num Ln ---')

# get Salesforce records to append
sfdc_df = sf.get_epicor_trans(creds=creds, order_num_ln=order_num_ln)
print(f'--- Got Matching SFDC Data: {sfdc_df.size} ---')

# add fo and asset
bookings_df = sf.add_sn_fo(creds=creds, df=bookings_df)
bookings_df = sf.add_sn_asset(creds=creds, df=bookings_df)
print(f'--- Added FO & SN: {bookings_df.size} ---')

--- Compiled Order Num Ln ---
--- Got Matching SFDC Data: 0 ---
--- Added FO & SN: 213026 ---


In [9]:
bookings_df.shape

(4631, 46)

In [10]:
# add cp and sales lead
bookings_df = sf.add_code_ref(creds=creds, df=bookings_df)
print(f'--- Added CP & SL: {bookings_df.size} ---')

--- Added CP & SL: 222288 ---


In [11]:
bookings_df.shape

(4631, 48)

In [12]:
# merge DataFrames
merged_df = epicor.merge_dfs(bookings_df, sfdc_df)
print(f'--- Merged DataFrames: {merged_df.size} ---')

--- Merged DataFrames: 219552 ---


In [16]:
merged_df.shape

(4574, 48)

In [17]:
merged_df

,OrderNum__c,OrderLn__c,Name,Book_Date__c,BookType__c,OrderRel__c,Prod_Code__c,SN__c,Order_Line_Amt__c,PO_Num__c,...,Packager__c,Market_Segment__c,Payment_Terms__c,OrderHed_FOB__c,SysRowID__c,Order_Num_Ln__c,Factory_Order__c,Asset__c,Sales_Lead__c,Channel_Partner__c
0,48586,9,762654,2020-03-19,Void,1,AEHC,B5EQ1E078,0.0,9300000815,...,FSE,AIRSEP,PROGRESS PMT 30,Ex-Works,874d3bbd-b8ea-4f3e-9768-33c1085f35a3,48586-9,a1ni0000001bF0hAAE,NaN,005i0000002OS1fAAG,001i000000LRvQxAAL
1,48633,1,759073,2020-01-21,Void,1,AP7I,B5EX2E079,-375043.0,13516,...,FSE,INDUS,PROGRESS PMT 90,Ex-Works Export,7762a8f1-bb02-4299-9737-9f4930e30ffb,48633-1,a1ni0000001bI0XAAU,NaN,005i0000002OS1hAAG,001i000000LRvQaAAL
2,53842,7,759240,2020-01-22,New,1,AIFC,B6EL1E142,0.0,4500004343,...,FSES,ENG,PROGRESS PMT 30,FOB East Coast,087959d1-06d0-454f-8260-e15bf84c29c8,53842-7,a1ni000000405ilAAA,NaN,005i0000006qn6WAAQ,001i000000LRvRLAA1
3,56311,8,757787,2020-01-03,Del,0,AICC,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,8857d704-1009-4474-8f7a-2f4403303ee2,56311-8,NaN,NaN,NaN,NaN
4,56311,12,757851,2020-01-03,Del,0,AICC,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,14fbdc8f-5744-40cb-8684-7ad5ce838e5f,56311-12,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4569,68701,2,763683,2020-03-31,New,1,PIDE,A8EV2118,429.8,1230,...,NaN,ENG,NET 30 DAYS,Ex-Works Export,8b9e721a-3d73-42c2-b7e1-eb4f95ed4e83,68701-2,a1ni0000000JDlsAAG,02ii0000006PujgAAC,005i0000006YNDBAA4,001i000001zk5sVAAQ
4570,68701,3,763684,2020-03-31,New,1,PIDE,A8EV2118,134.4,1230,...,NaN,ENG,NET 30 DAYS,Ex-Works Export,05db068d-f5c4-444a-833b-057a67fe4be4,68701-3,a1ni0000000JDlsAAG,02ii0000006PujgAAC,005i0000006YNDBAA4,001i000001zk5sVAAQ
4571,68701,4,763685,2020-03-31,New,1,PIDE,A8EV2118,41.3,1230,...,NaN,ENG,NET 30 DAYS,Ex-Works Export,b2042189-c837-49dc-b390-ed8463d25c18,68701-4,a1ni0000000JDlsAAG,02ii0000006PujgAAC,005i0000006YNDBAA4,001i000001zk5sVAAQ
4572,68701,5,763686,2020-03-31,New,1,PIDE,A8EV2118,177.1,1230,...,NaN,ENG,NET 30 DAYS,Ex-Works Export,9d63a65c-50e8-4bae-9c73-93c61eb0f40a,68701-5,a1ni0000000JDlsAAG,02ii0000006PujgAAC,005i0000006YNDBAA4,001i000001zk5sVAAQ


In [18]:
# upsert data
if merged_df.size > 0:
    sf.upsert_data(
        creds=creds, object_api='Epicor_Transaction__c',
        ex_id='Order_Num_Ln__c', df=merged_df
    )

3293 rows have been processed. 0.48 minutes have passed
4574 rows have been processed. 0.63 minutes have passed


In [ ]:
# move upserted files
for each in os.listdir(directory):
    if each == '.DS_Store' or each == 'Uploaded':
        continue
    shutil.move(
        os.path.join(directory, each),
        os.path.join(f'{directory}/Uploaded', each)
    )

print('--- DONE! ---')